# Loading and Formating Data pipeline

## Packages

In [2]:
%load_ext autoreload
%autoreload 2

In [39]:
import pandas as pd
import numpy as np
from utils import data_load
from dateutil.parser import parse

## Data Wrangling of CMU dataset

### Raw Metadata Extraction

We extract here into dataframe the raw information from the CMU dataset. Note that we are merging the plot, when available, directly in the raw movie dataframe.

In [9]:
raw_movie_df = data_load.get_raw_movie_dataframe("../data/MovieSummaries/movie.metadata.tsv","../data/MovieSummaries/plot_summaries.txt")
raw_character_df = data_load.get_raw_character_dataframe("../data/MovieSummaries/character.metadata.tsv")

### Tables Creation

We create now the different tables according to the ER diagram we have designed.

##### Country table and relation

In [10]:
country_df, comes_from_df = data_load.create_entry_and_relation_table(raw_movie_df,"countries",
                                                           "country_name","movie_id")

##### Genre table and relation

In [11]:
genre_df, is_of_type_df = data_load.create_entry_and_relation_table(raw_movie_df,"genres",
                                                           "genre_name","movie_id")

##### Language table and relation

In [12]:
language_df, spoken_languages_df = data_load.create_entry_and_relation_table(raw_movie_df,"languages","language_name",
                                                                  "movie_id",filter_dict={" language":""})

##### Character table

We remove characters with no Freebase_id because none of them has a name

In [13]:
character_df = pd.DataFrame(raw_character_df["character_name"][~raw_character_df.index.isna()])
character_df = character_df[~character_df.index.duplicated()]
character_df.index = character_df.index.rename("character_id")

##### Actor table

We will not keep actors and actresses that have a nan id, that are filled with only nan values or duplicated entries in the dataset.

One example of such duplicated entries is Clark Kent and Superman that are two different characters, thus there will be two rows in the original CMU character df, but they are played by the same actor.

In [14]:
actor_df = raw_character_df[["actor_name","actor_gender",
                                "actor_height","actor_ethnicity","actor_birth_date",
                                "freebase_actor_id"]].set_index("freebase_actor_id")
actor_table_columns_mapping = {"actor_birth_date":"birth_date","actor_gender":"gender",
                                "actor_height":"height","actor_ethnicity":"ethnicity",
                                "actor_name":"name","freebase_actor_id":"actor_id"}
actor_df = actor_df[~actor_df.index.isna()]
actor_df = actor_df[~actor_df.index.duplicated()].rename(
                columns=actor_table_columns_mapping)
actor_df.index = actor_df.index.rename("actor_id")
actor_df = actor_df.dropna(how="all")

##### Movie table

In [15]:
movie_df = raw_movie_df[["name","release_date","revenue","runtime","freebase_id","plot"]]
movie_df.index = raw_movie_df.index.rename("movie_id")

##### "Belongs to" table

In [16]:
belongs_to_df = pd.DataFrame(
    raw_character_df["wikipedia_movie_id"][~raw_character_df.index.isna()])
belongs_to_df = belongs_to_df.reset_index().drop_duplicates()
# Convert back index to linear range
belongs_to_df = belongs_to_df.reset_index()[["freebase_character_id","wikipedia_movie_id"]]
belongs_to_table_columns_mapping = {"freebase_character_id":"character_id","wikipedia_movie_id":"movie_id"}
belongs_to_df = belongs_to_df.rename(columns=belongs_to_table_columns_mapping)

##### "Plays" table

In [17]:
play_df = pd.DataFrame(
    raw_character_df[["freebase_actor_id","freebase_map_id"]][~raw_character_df.index.isna()])
play_df = play_df.reset_index().drop_duplicates()
# Convert back index to linear range
play_df = play_df.reset_index()[["freebase_actor_id","freebase_character_id","freebase_map_id"]]
play_table_columns_mapping = {"freebase_character_id":"character_id",
                                    "freebase_actor_id":"actor_id"}
play_df = play_df.rename(columns=play_table_columns_mapping)

##### "Appears in" table

Remove duplicates that can appear in the dataset (same actor for clark kent and superman)

In [18]:
appears_in_df = raw_character_df.reset_index()[["freebase_actor_id",
    "wikipedia_movie_id","actor_age_at_release_date"]].drop_duplicates()
# Convert back index to linear range
appears_in_table_columns_mapping = {"wikipedia_movie_id":"movie_id",
                                    "freebase_actor_id":"actor_id",
                                    "actor_age_at_release_date":"actor_age"}
appears_in_df = appears_in_df.rename(columns=appears_in_table_columns_mapping)

### Filter out the duplicated actors and actresses

In the dataset we have some duplicated actors and actresses. They have different freebase ids but have exactly the same attributes and are indeed duplicates when we look at the filmography. However, for many of such duplicates we do not have enough information to be assume with confidence that there are duplicates. Are two actors named John Bravo the same actors or not? It is hard to tell. Thus we decided that we tagged two actors entries as duplicates if they share the same name and same birthdate (the same birthyear is not consider as sufficient).

In [19]:
data_load.process_duplicated_actors(actor_df,[play_df,appears_in_df])

In [14]:
country_df.to_pickle("../data/generated/country_df.pkl")
comes_from_df.to_pickle("../data/generated/comes_from_df.pkl")
genre_df.to_pickle("../data/generated/genre_df.pkl")
is_of_type_df.to_pickle("../data/generated/is_of_type_df.pkl")
language_df.to_pickle("../data/generated/language_df.pkl")
spoken_languages_df.to_pickle("../data/generated/spoken_languages_df.pkl")
character_df.to_pickle("../data/generated/character_df.pkl")
actor_df.to_pickle("../data/generated/actor_df.pkl")
movie_df.to_pickle("../data/generated/movie_df.pkl")
belongs_to_df.to_pickle("../data/generated/belongs_to_df.pkl")
play_df.to_pickle("../data/generated/play_df.pkl")
appears_in_df.to_pickle("../data/generated/appears_in_df.pkl")

---

## Data Integration

Our primary goal is to perform a time serie analysis of the different features we have in our movie dataset. The problem is that around 12% of the dataset is missing the release date entry. We will try to gather information from IMDB and Wikipedia to recover the information and thus avoid to throw away this data.

### Wikipedia Data Integration

In [120]:
wikipedia_data = pd.read_json("../data/Wikipedia/no_release_date_movies.json").T

In [121]:
wikipedia_data[["Release dates","Release date","Original release"]].head(20)

,Release dates,Release date,Original release
11250635,NaN,"[November 28, 1941]",NaN
31137877,NaN,[5 September 1927],NaN
27374355,NaN,"[5 October 1977, (, 1977-10-05]",NaN
29766415,NaN,"[June 29, 1945, (, 1945-06-29]",NaN
28415406,NaN,"[2007, (, 2007]",NaN
21373150,NaN,"[December 10, 1982, (, 1982-12-10]",NaN
11717027,NaN,"[March 24, 1964, (, 1964-03-24]",NaN
10628777,NaN,[15 June 1995],NaN
18902861,NaN,NaN,"[March 6, 1988, (, 1988-03-06, (U.S.)]"
25790807,NaN,"[October 12, 2009, (, 2009-10-12, (Los Angel...",NaN


In [122]:
clean_date(wikipedia_data)

In [123]:
wikipedia_data[["Release dates","Release date","Original release"]].head(20)

,Release dates,Release date,Original release
11250635,[nan],"[November 28, 1941]",[nan]
31137877,[nan],[5 September 1927],[nan]
27374355,[nan],"[5 October 1977, 1977-10-05]",[nan]
29766415,[nan],"[June 29, 1945, 1945-06-29]",[nan]
28415406,[nan],[2007],[nan]
21373150,[nan],"[December 10, 1982, 1982-12-10]",[nan]
11717027,[nan],"[March 24, 1964, 1964-03-24]",[nan]
10628777,[nan],[15 June 1995],[nan]
18902861,[nan],[nan],"[ U.S., 1988-03-06, March 6, 1988]"
25790807,[nan],"[October 12, 2009, 2009-10-12, Los Angeles La...",[nan]


In [124]:
def clean_date(wikipedia_dataframe: pd.DataFrame,
               date_columns=["Release dates","Release date","Original release"]):
    for col in date_columns:
        wikipedia_dataframe[col] = wikipedia_dataframe[col].apply(lambda d: clean_date_entry(d))
        
def clean_date_entry(date_field: list[str]) -> str:
    if type(date_field) != list:
        date_field = [str(date_field)]
    clean_date = list(set([s.replace("(","").replace(")","") for s in date_field if s != '\xa0(']))
    return clean_date

def parse_date(date_field: list[str]) -> str:
    for entry in date_field:
        if "-" in entry:
            return entry
    if len(date_field) == 1:
        try:
            date_parsed = parse(date_field[0])
            format_length = len(date_field.split(" "))
            if format_length == 1:
                date_parsed.strftime("%Y")
            elif format_length == 2:
                date_parsed.strftime("%Y/%m")
            elif format_length == 3:
                date_parsed.strftime("%Y/%m/%d")
            else:
                return ""

In [128]:
"123".split(" ")

['123']

In [127]:
a = parse("1021")
a.strftime()

datetime.datetime(1021, 11, 16, 0, 0)

### Combine CMU with IMDB

In [4]:
df_imdb = data_load.load_imdb_title_basics()
df_imdb = df_imdb[df_imdb.title_type == "movie"]
df_imdb.drop('original_title', axis='columns', inplace=True)

In [25]:
# we merge on the titles name (problem: names not unique and maybe spelling errors)
merged = movie_df.merge(df_imdb, left_on='name', right_on='primary_title', how='left', suffixes=('_cmu', '_imdb'))

In [26]:
merged

,name,release_date,revenue,runtime,freebase_id,plot,tconst,title_type,primary_title,is_adult,start_year,end_year,runtime_minutes,genres
0,Ghosts of Mars,2001-08-24,14010832.0,98.0,/m/03vyhn,"Set in the second half of the 22nd century, th...",tt0228333,movie,Ghosts of Mars,0,2001,<NA>,98,"[Action, Horror, Sci-Fi]"
1,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,/m/08yl5d,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
2,Brun bitter,1988,NaN,83.0,/m/0crgdbh,NaN,tt0094806,movie,Brun bitter,0,1988,<NA>,83,"[Crime, Drama]"
3,White Of The Eye,1987,NaN,110.0,/m/0285_cd,A series of murders of rich young women throug...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
4,A Woman in Flames,1983,NaN,106.0,/m/01mrr1,"Eva, an upper class housewife, becomes frustra...",tt0083949,movie,A Woman in Flames,0,1983,<NA>,106,[Drama]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163487,Mermaids: The Body Found,2011-03-19,NaN,120.0,/m/0j7hxnt,Two former National Oceanic Atmospheric Admini...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
163488,Knuckle,2011-01-21,NaN,96.0,/m/0g4pl34,{{No plot}} This film follows 12 years in the ...,tt1606259,movie,Knuckle,0,2011,<NA>,97,"[Biography, Documentary, Drama]"
163489,Another Nice Mess,1972-09-22,NaN,66.0,/m/02pygw1,NaN,tt0362411,movie,Another Nice Mess,0,1972,<NA>,66,[Comedy]
163490,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,/m/03pcrp,"The story takes place in the year 2092,The Sup...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN


In [20]:
merged = merged[merged.release_date.apply(lambda r: r.year) == merged.start_year]

In [21]:
# we have duplicate entries
duplicates = merged.freebase_id.value_counts()
duplicates = duplicates[duplicates > 1]
duplicates

/m/0h1z21s    5
/m/064p159    4
/m/0bbx0_     4
/m/09vq1kn    4
/m/03cg7t2    3
             ..
/m/03cb5j0    2
/m/04g0gvz    2
/m/07scm0y    2
/m/06w9zfg    2
/m/09s60jk    2
Name: freebase_id, Length: 465, dtype: Int64

In [22]:
df_imdb_rating = data_load.load_imdb_title_ratings()

In [23]:
# keep the ones with the most imdb votes => most meaningful
df = merged[merged.freebase_id.isin(list(duplicates.index))].merge(df_imdb_rating, on='tconst', how='left')

In [24]:
dropping = []
for id in list(duplicates.index):
    dft = df[df.freebase_id == id]
    ids = list(dft.sort_values('num_votes', ascending=False).iloc[1:].tconst.values)
    dropping += ids

In [25]:
merged = merged[~merged.tconst.isin(dropping)]

In [26]:
merged.head()

,wikipedia_movie_id,freebase_id,movie_name,release_date,revenue,runtime,languages,countries,genres_cmu,tconst,title_type,primary_title,is_adult,start_year,end_year,runtime_minutes,genres_imdb
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...",tt0228333,movie,Ghosts of Mars,0,2001,<NA>,98,"[Action, Horror, Sci-Fi]"
2,28463795,/m/0crgdbh,Brun bitter,1988-01-01,NaN,83.0,[Norwegian Language],[Norway],"[Crime Fiction, Drama]",tt0094806,movie,Brun bitter,0,1988,<NA>,83,"[Crime, Drama]"
4,261236,/m/01mrr1,A Woman in Flames,1983-01-01,NaN,106.0,[German Language],[Germany],[Drama],tt0083949,movie,A Woman in Flames,0,1983,<NA>,106,[Drama]
9,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,[English Language],[United States of America],"[Musical, Comedy, Black-and-white]",tt0029852,movie,Alexander's Ragtime Band,0,1938,<NA>,106,"[Drama, Music, Musical]"
10,9997961,/m/06_y2j7,Contigo y aquí,1974-01-01,NaN,0.0,[Spanish Language],[Argentina],"[Musical, Drama, Comedy]",tt0200545,movie,Contigo y aquí,0,1974,<NA>,70,"[Comedy, Drama, Musical]"


In [27]:
merged.to_pickle('../data/generated/movie_metadata.pkl')